-------------
Aviachipta narxini bashorat qilish
-----------

In [182]:
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

Ushbu maʼlumotlar toʻplami quyidagilarni o'z ichiga oladi:

Tarkib:

`ID`: Ketma - ketlik uchun qo'yilgan sonlar.

`Airline`: Parvoz qilingan aviakompaniya nomi.

`Flight`: Parvoz qilingan ID raqami.

`Source_city`: Parvoz qaysi shahardan boshlanishi.

`Departure_time`: Samolyotning ketish vaqti.

`Stop`: Parvoz davomida to'xtashlar soni.

`Arrival_time`: Qaytish vaqti.

`Destination_city`: Qaysi davlatga parvoz qilinayotgani.

`Class`: Foydalanuvchi samolyotning qaysi klass turidan foydalanib uchganligi.

`Duration`: Parvoz davomiyligi.

`Days_left`: Samolyotning qancha vaqtdan keyin qaytishi.

In [160]:
df1 = pd.read_csv("/content/sample_solution.csv")
test_set = pd.read_csv("/content/test_data.csv")
train_set = pd.read_csv("/content/train_data.csv")

`Sample_Solution` datasetini `df1`ga saqlaymiz

In [161]:
df1.head()

,id,price
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [162]:
test_set.head()

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
0,1,Air_India,AI-765,Kolkata,Evening,one,Night,Delhi,Business,28.25,2
1,2,Vistara,UK-747,Delhi,Early_Morning,one,Night,Mumbai,Business,13.83,34
2,3,Air_India,AI-570,Mumbai,Early_Morning,zero,Early_Morning,Chennai,Business,2.00,30
3,4,AirAsia,I5-974,Hyderabad,Night,one,Late_Night,Delhi,Economy,5.17,26
4,5,Air_India,AI-770,Kolkata,Night,one,Afternoon,Mumbai,Economy,16.33,35


In [163]:
train_set.head()

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
1,2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
2,3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
3,4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
4,5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712


Eng asosiy qiliinadigan ishlardan biri bu jadvalni matnlarni mos ravishda son ko'rinishiga o'tkazish kerak.

Chunki matnlar bilan ishlab bo'lmaydi.

Bu yerda turli-xil modellar bor, Masalan: LabelEncoder(), OneHotEncoder(), StandardScaler(), MinMaxScaler() va boshqalar.

Bularni orasidan bizga natijani yuqori beradigan modelni tanlab olishimiz kerak.

`StandardScaler` - bilan qilganimda `Public Score` 6000 - 7000 orasida natija bergan.
`RandomForestRegerssor` - orqali esa 4000-5000 oraliqda natija chiqdi.

Lekin `LabelEncoder` - orqali esa 3500 dan kam natija chiqdi. bizga aynan 0 ga yaqin natija chiqishi kerak va biz shu modelni tanlab loyihamizga ishlatamiz

--------------
Pastdagi kodda `train_set` jadvalidagi matn ko'rinishida bo'lgan ustunlarni `labelencoder` orqali sonlar ko'rinishiga o'tkazib olamiz

In [164]:
label_encoder = LabelEncoder()
train_set['airline'] = label_encoder.fit_transform(train_set['airline'])
train_set['source_city'] = label_encoder.fit_transform(train_set['source_city'])
train_set['destination_city'] = label_encoder.fit_transform(train_set['destination_city'])
train_set['class'] = label_encoder.fit_transform(train_set['class'])
train_set['stops'] = label_encoder.fit_transform(train_set['stops'])
train_set['class'] = label_encoder.fit_transform(train_set['class'])
train_set['departure_time'] = label_encoder.fit_transform(train_set['departure_time'])
train_set['arrival_time'] = label_encoder.fit_transform(train_set['arrival_time'])

Natijani ko'ramiz:

In [165]:
train_set.head()

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,1,5,UK-810,0,1,0,5,5,1,14.25,21,7212
1,2,4,SG-5094,3,2,2,5,4,1,1.75,7,5292
2,3,5,UK-846,0,4,0,2,2,0,9.58,5,60553
3,4,5,UK-706,4,4,0,2,3,1,6.75,28,5760
4,5,3,6E-5394,1,1,2,4,5,1,2.00,4,10712


Endi esa `test_set` datasetni ham sonlar ko'rinishiga o'tkazamiz

In [166]:
test_set['airline'] = label_encoder.fit_transform(test_set['airline'])
test_set['source_city'] = label_encoder.fit_transform(test_set['source_city'])
test_set['destination_city'] = label_encoder.fit_transform(test_set['destination_city'])
test_set['class'] = label_encoder.fit_transform(test_set['class'])
test_set['stops'] = label_encoder.fit_transform(test_set['stops'])
test_set['class'] = label_encoder.fit_transform(test_set['class'])
test_set['departure_time'] = label_encoder.fit_transform(test_set['departure_time'])
test_set['arrival_time'] = label_encoder.fit_transform(test_set['arrival_time'])

Modelimizga `price` bashorat qilinishi kerak bo'lgan ustunni olib tashlab qolgan qiymatlarni yangi `X_train` ga saqlaymiz va `price` ustunini alohida saqlab olamiz

In [168]:
X_train = train_set.drop('price', axis=1)
Y_train = train_set['price']

Modelimizni o'qitish uchun `train_set` ni qismlarga ajratib olamiz

In [169]:
train_set1, test_set1 = train_test_split(train_set, test_size=0.2, random_state=42)

datasetmizni bitta massiv ko'rinishiga keltirish uchun pipline dan foydalanamiz.

`num_pipeline` - bu matn ko'rinishidagi ustunlarni sonlarga o'tkazganimizdagi qiymatlarni standartlashtirib olishimiz kerak.

`Standartlashtirish` - agarda malumotlar to'plami normal taqsimlangan bo'lsa(```Gauss taqsimoti```), unda standartlashtirishni tanlagan ma'qul.

`cat_pipeline` bu 'flight' ustunida ko'p takrorlanga qiymatlar bor , ularni boshqacha usulda tahrirlaymiz.

`full_pipeline` shu qiymatlarni birlashtirib yakka holga olib kelamiz

In [170]:
num_attribs=['airline', 'stops', 'class', 'duration', 'days_left', 'source_city', 'departure_time', 'arrival_time', 'destination_city']
cat_attribs=['flight']

num_pipeline = Pipeline([
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', cat_pipeline, cat_attribs)
])

Endi tepadagi `X_train` datasetini massiv ko'rinishiga o'tkazamiz

In [171]:
X_predict = full_pipeline.fit_transform(X_train)
X_predict.toarray()[0:5,:]

array([[ 1.03113236, -0.4306786 ,  0.67396452, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.48472127,  2.55289329,  0.67396452, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.03113236, -0.4306786 , -1.48375764, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.03113236, -0.4306786 ,  0.67396452, ...,  0.        ,
         0.        ,  0.        ],
       [-0.06168981,  2.55289329,  0.67396452, ...,  0.        ,
         0.        ,  0.        ]])

X_test dataseini ham massivga o'tkazamiz

In [174]:
X_test=test_set.copy()
# Y_test=test_set['price'].copy()
X_test_predict = full_pipeline.transform(X_test)

Biz ```RandomForestRegressor()``` modelidan foydalandik, bu model `LinearRegression()` modelidan yaxshiroq natijani bera oladi

`X_train` jadvalini massivga o'tkazganimiz bilan 'price' qiymati saqlangan qiymatlarni birgalikda model orqali moslab modelni o'qitamiz...

In [175]:
RF_model = RandomForestRegressor()
RF_model.fit(X_predict, Y_train)

RandomForestRegressor()

Endi `X_predict` o'rniga `X_test_predict` massivini berib `RF_model`ni o'qitib olamiz

In [176]:
Y_predicted = RF_model.predict(X_test_predict)

tepadagi o'qitilgan modelimizga `price` ustuni yo'q bo'lgan `test_set`imizni massiv ko'rinishiga aylantirib beramiz, va oxirida predict qilib `price` ustunini bashorat qilamiz

In [177]:
test_predict = full_pipeline.transform(test_set)
predict_price = RF_model.predict(test_predict)

biz bashorat qilgan `price` ustunini ``df1`` jadvaliga saqlab olamiz

In [179]:
df1['price'] = predict_price
df1

,id,price
0,1,54152.46
1,2,65844.30
2,3,23321.24
3,4,2267.35
4,5,5540.24
...,...,...
4995,4996,60635.89
4996,4997,4498.97
4997,4998,50271.00
4998,4999,50504.09


biz bashorat qilgan price ustuni haqida malumot

In [180]:
df1.describe()

,id,price
count,5000.000000,5000.000000
mean,2500.500000,20521.888790
std,1443.520003,22295.746939
min,1.000000,1303.730000
25%,1250.750000,4908.147500
50%,2500.500000,7253.530000
75%,3750.250000,42614.175000
max,5000.000000,84185.210000


Va oxirida bizga kerakli jadvalni taxlab olganimizdan so'ng shu jadvalni `.csv` formatida kompyuterimizga yuklab olamiz...

In [181]:
df1.to_csv('Sample_solution(4).csv', index=False)